In [1]:
from option_pricers.pricing import PriceModel
from option_pricers.black import black76_torch
import torch

import pandas as pd 
import numpy as np
import os
import yaml
from utils.MAPPING import *
from position.flat import *

In [2]:
# split dataframe by columns whose names contain a string
def split_dataframe(df, col_str):
    cols = df.columns
    cols = cols[cols.str.contains(col_str)]
    cols = cols.union(["Dates"])
    return df[cols]

In [3]:
asset = yaml.load(open('utils/asset.yml'), Loader=yaml.FullLoader)

In [4]:
pricing = pd.read_csv('data/allConts.csv')
pricing.loc[:,'Dates'] = pd.to_datetime(pricing.Dates,format="%d/%m/%Y")
pricing = pricing.tail(3600)

In [5]:
p_cols = pricing.columns.str[:2].unique()
p_cols = p_cols.drop('Da')
assets = [split_dataframe(pricing, _) for _ in p_cols]
assets = dict(zip(p_cols,assets))

In [6]:
assets['QC'].head()

,Dates,QC1 Comdty,QC10 Comdty,QC2 Comdty,QC3 Comdty,QC4 Comdty,QC5 Comdty,QC6 Comdty,QC7 Comdty,QC8 Comdty,QC9 Comdty
2560,2009-10-26 00:00:00,2175,2135,2208,2234,2166,2151,2135,2117,2106,2121
2561,2009-10-27 00:00:00,2172,2126,2207,2236,2163,2146,2126,2108,2097,2112
2562,2009-10-28 00:00:00,2153,2108,2186,2216,2145,2129,2108,2094,2079,2094
2563,2009-10-29 00:00:00,2147,2097,2182,2208,2137,2117,2099,2083,2068,2083
2564,2009-10-30 00:00:00,2127,2092,2168,2191,2120,2100,2081,2068,2063,2078


In [7]:
dicts = {
    'QC':
        {
            'LOTS': [100,500,-150,0,580],
            'CONTS': ['U23','Z23','H24','K24','N24'],
            'MULTIPLIER': 10,
            'TICK_SIZE': 1,
        },
    'CC':
        {
            'LOTS': [0,-423,-298],
            'CONTS': ['U23','Z23','H24'],
            'MULTIPLIER': 10,
            'TICK_SIZE': 1,
        },
     
    'RC':
        {
            'LOTS': [15,-30,-34,27,6],
            'CONTS': ['U23','X23', 'F24','H24','K24'],
            'MULTIPLIER': 10,
            'TICK_SIZE': 1,
        },
  
    'KC':
        {
            'LOTS': [0,17,67,12,1],
            'CONTS': ['U23','Z23','H24','K24','N24'],
            'MULTIPLIER': 37500,
            'TICK_SIZE': 0.01,
        },
    'SB':
        {
            'LOTS': [181,-304,-682,2448,-217,-422,255,218,-46],
            'CONTS': ['V23','H24','K24','N24','V24','H25','K25','N25','V25'],
            'MULTIPLIER': 112000,
            'TICK_SIZE': 0.01,
        },
    'QW':{
            'LOTS': [0,818,217,-719,611,-281,371,16],
            'CONTS': ['V23','Z23','H24','K24','Q24','V24','Z24', 'H25'],
            'MULTIPLIER': 50,
            'TICK_SIZE': 1,
            
    }
}

In [8]:
dicts = {
    # 'QC':
    #     {
    #         'LOTS': [100,500,-150,0,580],
    #         'CONTS': ['U23','Z23','H24','K24','N24'],
    #         'MULTIPLIER': 10,
    #         'TICK_SIZE': 1,
    #     },
    'CC':
        {
            'LOTS': [0,-423,-298],
            'CONTS': ['U23','Z23','H24'],
            'MULTIPLIER': 10,
            'TICK_SIZE': 1,
        },
        
    # 'KC':
    #     {
    #         'LOTS': [0,17,67,12,1],
    #         'CONTS': ['U23','Z23','H24','K24','N24'],
    #         'MULTIPLIER': 37500,
    #         'TICK_SIZE': 0.01,
    #     },
    'SB':
        {
            'LOTS': [181,-304,-682,2448,-217,-422,255,218],
            'CONTS': ['V23','H24','K24','N24','V24','H25','K25','N25'],
            'MULTIPLIER': 112000,
            'TICK_SIZE': 0.01,
        },
    'QW':{
            'LOTS': [0,818,217,-719,611,-281,371],
            'CONTS': ['V23','Z23','H24','K24','Q24','V24','Z24'],
            'MULTIPLIER': 50,
            'TICK_SIZE': 1,
            
    }
}

In [9]:
# from random import randint
# random_portfolios = [[randint(-200,200) for _ in range(5)] for _ in assets]
# random_portfolios[0]

# portfolios = random_portfolios

In [10]:
assets['KC'].loc[:,'KC1 Comdty'].pct_change().dropna()[:-100].mean()

0.00028489625834792387

In [11]:
0.00028489625834792387 * 3000

0.8546887750437716

In [52]:
dfs = assets.copy()

fl=flat()

pR = []
pS = []
pP = []
pF = []
mR = []
mL = []
vst = []
cVar = []


for k,v in dicts.items():
    
    conts = v['CONTS']
    conts = [f'{k} {_}' for _ in conts]
    lot_multiplier = v['MULTIPLIER']*v['TICK_SIZE']
    lots = np.multiply(v['LOTS'],lot_multiplier)
    
        
    rels = []
    for i in range(len(lots)):
        rels.append(f'{k}{i+1} Comdty')
    rels = ['Dates'] + rels
    
    dfs[k] = dfs[k][rels]
    
    dfs[k].set_index('Dates', inplace=True)
    start = dfs[k].copy()
    dfs[k] = dfs[k].pct_change()
    dfs[k].dropna(inplace=True)
    
    dfs[k].columns = conts
    
    # start  = start[-1:]
    # start.columns = conts
    # P_S = start.copy()

    # vStart = start.copy()
    
    # dict_conts = dict(zip(conts,lots))
    
    
    last_p  = start[-1:]
    last_p.columns = conts
    # P_S = last_p.copy()

    # vStart = start.copy()

    dict_conts = dict(zip(conts,lots))

    start = last_p.copy()
    start = start.apply(lambda x: x * dict_conts[x.name])
    vStart = start.copy()
    
    vst.append(vStart)
    
    # print(dict_conts)
    # print(dfs[k].tail(1))
    
    returns, weights, weightsA, weights_adjuster, meanReturns, covMatrix = fl.returns_weights(dfs[k],dict_conts)
    portResults, portfolio_sims, initialPortfolio = fl.MCsimulation(100_000,1,vStart,weights_adjuster,lot_multiplier,weightsA,meanReturns,covMatrix)

    pR.append(portResults)
    pS.append(portfolio_sims)
    pP.append(initialPortfolio)
    pF.append(initialPortfolio - portResults)
    
    cVar.append(initialPortfolio-fl.mcCVaR(portResults,0.95))
    
    
    print('--------------+++++++++++++++++----------------')
    print(k)
    print('Portfolio Value:')
    print(initialPortfolio)
    print('Conditional VaR, 99%:')    
    print("{:,}".format(round(initialPortfolio-fl.mcCVaR(portResults,0.99))))
    print('VaR, 99%:')    
    print("{:,}".format(round(initialPortfolio-fl.mcVaR(portResults,0.99))))
    
    mR.append(dfs[k])
    mL.append(dict_conts)
    
dVar = dict(zip(dicts.keys(),cVar))
    
    

--------------+++++++++++++++++----------------
CC
Portfolio Value:
24202240
Conditional VaR, 99%:
971,131
VaR, 99%:
848,559
--------------+++++++++++++++++----------------
SB
Portfolio Value:
117164824.0
Conditional VaR, 99%:
1,363,099
VaR, 99%:
1,182,339
--------------+++++++++++++++++----------------
QW
Portfolio Value:
97206595.0
Conditional VaR, 99%:
1,137,084
VaR, 99%:
998,971


In [53]:
mRet = pd.concat(mR,axis=1)
masterPosition = {}
for d in mL:
    masterPosition.update(d)
vStart = pd.concat(vst,axis=1)

In [55]:

returns, weights, weightsA, weights_adjuster, meanReturns, covMatrix = fl.returns_weights(mRet,masterPosition)
portResults, portfolio_sims, initialPortfolio = fl.MCsimulation(100_000,1,vStart,weights_adjuster,lot_multiplier,weightsA,meanReturns,covMatrix)

pR.append(portResults)
pS.append(portfolio_sims)
pP.append(initialPortfolio)
pF.append(initialPortfolio - portResults)

cVar.append(initialPortfolio-fl.mcCVaR(portResults,0.99))





In [56]:
print('--------------+++++++++++++++++----------------')
print('MASTER')
print('Portfolio Value:')
print(initialPortfolio)
print('Conditional VaR, 99%:')    
print("{:,}".format(round(initialPortfolio-fl.mcCVaR(portResults,0.99))))
print('VaR, 99%:')    
print("{:,}".format(round(initialPortfolio-fl.mcVaR(portResults,0.99))))

--------------+++++++++++++++++----------------
MASTER
Portfolio Value:
238573659.0
Conditional VaR, 99%:
2,746,849
VaR, 99%:
2,400,074


In [57]:
cVar_dict = dict(zip([_ for _ in dicts.keys()] + ['Master'], cVar))
pOffset = np.sum([p for i,p in dVar.items()]) - cVar_dict['Master']
round(pOffset)

740796

In [45]:
pReturns = pd.concat(pF, axis=1)
pReturns.columns = [_ for _ in dicts.keys()] + ['Master']

In [46]:
pReturns = pReturns.melt()
pReturns.columns = ['portfolio','sLoss']
# group by portfolio and add column for percentile
pReturns['percentile'] = pReturns.groupby('portfolio').sLoss.rank(pct=True)

In [47]:

pReturns.percentile = round(pReturns.percentile * 100,1)


In [48]:
aggreRet = pReturns.groupby(['portfolio','percentile']).mean('sLoss')
aggreRet.reset_index(inplace=True)

In [49]:
pReturns.to_csv('data/pDemoReturns2_1day.csv')



In [50]:
aggreRet.to_csv('data/aggreRetDemo21d.csv', index=False)

In [ ]:
option = {
            'S_0': 1000., 
            'strike': 2500., 
            'time_to_expiry': .0, 
            'implied_vol': .27, 
            'dividend': 0.,
            'riskfree_rate': .035, 
            'option_sign': -1.
        }
bs = black76_torch(**option)
bs.premium()

In [125]:
vst[0].index[0] - 

Timestamp('2023-08-11 00:00:00')

In [127]:
date_exp = {
'Q23': pd.to_datetime('2023-08-20'),
'U23': pd.to_datetime('2023-09-20'),
'V23': pd.to_datetime('2023-10-20'),
'X23': pd.to_datetime('2023-11-20'),
'Z23':pd.to_datetime('2023-12-20'),
'F24': pd.to_datetime('2024-01-20'),
'G24': pd.to_datetime('2024-02-20'),
'H24': pd.to_datetime('2024-03-20'),
'J24': pd.to_datetime('2024-04-20'),
'K24': pd.to_datetime('2024-05-20'),
'M24': pd.to_datetime('2024-06-20'),
'N24': pd.to_datetime('2024-07-20')}

In [136]:
tte = dict(zip(date_exp.keys(), [((v - vst[0].index[0]).days/365) for k,v in date_exp.items() ]))

In [139]:
rf  = risk_surface()

In [142]:
assets['KC']

,Dates,KC1 Comdty,KC2 Comdty,KC3 Comdty,KC4 Comdty,KC5 Comdty,KC6 Comdty,KC7 Comdty,KC8 Comdty
2560,2009-10-26 00:00:00,136.50,139.45,141.35,143.00,144.65,147.05,149.45,151.05
2561,2009-10-27 00:00:00,136.35,139.30,141.20,142.85,144.50,146.95,149.30,151.00
2562,2009-10-28 00:00:00,134.60,137.55,139.40,141.10,142.85,145.45,147.90,149.60
2563,2009-10-29 00:00:00,136.75,139.75,141.65,143.30,145.05,147.65,150.25,151.95
2564,2009-10-30 00:00:00,135.50,138.55,140.45,142.10,143.80,146.45,148.95,150.70
...,...,...,...,...,...,...,...,...,...
6155,2023-08-07 00:00:00,163.95,163.45,164.60,165.45,166.10,166.95,168.20,169.00
6156,2023-08-08 00:00:00,161.35,160.85,162.00,162.95,163.65,164.45,165.70,166.50
6157,2023-08-09 00:00:00,160.75,160.20,161.35,162.25,162.90,163.55,164.65,165.35
6158,2023-08-10 00:00:00,159.90,159.65,160.90,161.85,162.40,162.95,164.05,164.90


In [184]:
df = assets['KC']
df = df.loc[:,df.columns != 'Dates']


rf.shock_map(returns,60)


ValueError: operands could not be broadcast together with shapes (9,25) (9,) 

In [180]:
       
        bumps = [-2,-1.5,-1,-0.5,0,0.5,1,1.5,2]
        shock_map = returns.copy()
        shock_map = shock_map[-30:].std()
        shock_map = np.array(shock_map)
        shock_map = shock_map.reshape(1,len(shock_map))
        shock_map = np.repeat(shock_map, len(bumps), axis=0)
        shock_map = shock_map.T*bumps
        shock_map = shock_map.T
        

In [193]:
    def shock_map(last_px, returns, window,bumps = [-2,-1.5,-1,-0.5,0,0.5,1,1.5,2]):
        shock_map = returns.copy()
        shock_map = shock_map[-window:].std()
        shock_map = shock_map.to_frame()
        shock_map.reset_index(inplace=True)
        shock_map = shock_map.rename(columns={'index': 'contract', 0: 'sdev'})
        dL = []
        for i, p in enumerate(shock_map.sdev):
            dL.append(dict(zip(bumps, [p*x for x in bumps])))
            
        dL = pd.DataFrame(dL)
        dL.index = shock_map.contract
        dL.reset_index(inplace=True)
        
        dL = dL.T
        dL.columns = dL.iloc[0]
        dL = dL.iloc[1:,:]
        dL= dL.apply(lambda x: (1+x) * last_px[x.name], axis=0)
                
        
        # for k,v in last_px.items():
        #     dL[dL.select_dtypes("number").columns].loc[:,dL.contract == k] *= v 
        #     dL[dL.select_dtypes("number").columns].loc[:,dL.contract == k] += v


        return shock_map, dL

In [194]:
vst[2]

,KC U23,KC Z23,KC H24,KC K24,KC N24
Dates,,,,,
2023-08-11,-2998125.0,21852093.75,-48270000.0,910406.25,5176500.0


In [195]:
s, l = shock_map(vStart,returns, 30)

KeyError: 'portfolio'